# MAIN ISSUE HERE:
The manner in which TARF_CD==59 was excluded
A premise can have multiple tariff codes.  Only removing entries where TARF_CD==59 left the other entries for the premise, which were not EV codes.
Therefore, this essentially had the effect of sneaking EV premises into the non-EV sample!

In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import GrubbsTest

# NOTES

- TARF_CD==59
    - submetered
    - the main meter is accounting for the EV meter as well
    - Both meters are fed from the same drop. 
    - If you wanted just the house load, you would need to subtract the EV meter. 
    - If the tariff says separately metered (don’t think I included any) they are fed off separate drops and the main meter does not account the EV

In [ ]:
apc_trff_df = pd.read_csv(r'C:\Users\s346557\Documents\LocalData\EVs\Tariff\apc_tarf_EV.csv')
imp_trff_df = pd.read_csv(r'C:\Users\s346557\Documents\LocalData\EVs\Tariff\imp_tarf_EV.csv')
oh_trff_df = pd.read_csv(r'C:\Users\s346557\Documents\LocalData\EVs\Tariff\oh_tarf_EV.csv')
pso_trff_df = pd.read_csv(r'C:\Users\s346557\Documents\LocalData\EVs\Tariff\pso_tarf_EV.csv')

In [ ]:
print(apc_trff_df.shape[0])
print(imp_trff_df.shape[0])
print(oh_trff_df.shape[0])
print(pso_trff_df.shape[0])

In [ ]:
opco_trff_dfs = dict(
    apc=apc_trff_df, 
    imp=imp_trff_df, 
    oh=oh_trff_df, 
    pso=pso_trff_df, 
)
#-------------------------
# Remove any entries where TARF_CD=59
# Add 'opco' column
# Make sure all PREM_NB values have 9 characters
for opco_i in opco_trff_dfs.keys():
    trff_df_i = opco_trff_dfs[opco_i]
    trff_df_i = trff_df_i[trff_df_i['TARF_CD']!=59].copy()
    #-----
    trff_df_i['opco'] = opco_i
    #-----
    trff_df_i['PREM_NB'] = trff_df_i['PREM_NB'].astype(str)
    trff_df_i['PREM_NB'] = trff_df_i['PREM_NB'].str.zfill(9)
    #-----
    opco_trff_dfs[opco_i] = trff_df_i

In [ ]:
save_dir_base = r'C:\Users\s346557\Documents\LocalData\EVs\Data'
expand_time = pd.Timedelta('60 days')

if not os.path.exists(save_dir_base):
    os.makedirs(save_dir_base)

In [ ]:
all_trff_dfs = pd.concat(list(opco_trff_dfs.values()))
#-----
all_trff_dfs_evs = all_trff_dfs[all_trff_dfs['EV']==1].copy()
all_trff_dfs_non = all_trff_dfs[all_trff_dfs['EV']==0].copy()
assert(all_trff_dfs.shape[0]==(all_trff_dfs_evs.shape[0]+all_trff_dfs_non.shape[0]))
#-------------------------
assert(all_trff_dfs_evs['PREM_NB'].nunique()==all_trff_dfs_evs['PREM_NB'].shape[0])
#-----
if all_trff_dfs_non['PREM_NB'].nunique()!=all_trff_dfs_non['PREM_NB'].shape[0]:
    all_trff_dfs_non = all_trff_dfs_non.sort_values(by=['TARF_EFCT_TS'], ascending=False).groupby(['PREM_NB'], as_index=False, group_keys=False).first()
assert(all_trff_dfs_non['PREM_NB'].nunique()==all_trff_dfs_non['PREM_NB'].shape[0])

#--------------------------------------------------
all_trff_dfs_evs['t_search_min'] = pd.to_datetime(all_trff_dfs_evs['TARF_EFCT_TS'])
all_trff_dfs_evs['t_search_max'] = all_trff_dfs_evs['t_search_min'] + pd.Timedelta(expand_time)
#-------------------------
# For non-EV entries, collect events from the general time period as EVs
# EVS_t_min = pd.to_datetime(all_trff_dfs_evs['TARF_EFCT_TS']).min()
# EVS_t_max = pd.to_datetime(all_trff_dfs_evs['TARF_EFCT_TS']).max()
# !!!!!!!!!!! Actually, cds_ds_db.ev_ami_final only contains data from 2022-04-01 to 2023-04-01!
EVS_t_min = pd.to_datetime('2022-04-01')
EVS_t_max = pd.to_datetime('2023-04-01')
#-----
all_trff_dfs_non[['t_search_min', 't_search_max']] = [
    Utilities_dt.get_random_datetime_interval_between(EVS_t_min, EVS_t_max, pd.Timedelta(expand_time)) 
    for _ in range(all_trff_dfs_non.shape[0])
]
#--------------------------------------------------
all_trff_dfs = pd.concat([all_trff_dfs_evs, all_trff_dfs_non])

# So, I guess there are some premises with EV and non-EV tariffs.
# If this is the case, keep only the EV tariff
if all_trff_dfs['PREM_NB'].nunique()!=all_trff_dfs['PREM_NB'].shape[0]:
    all_trff_dfs = all_trff_dfs.sort_values(by=['EV'], ascending=False).groupby(['PREM_NB'], as_index=False, group_keys=False).first()
assert(all_trff_dfs['PREM_NB'].nunique()==all_trff_dfs['PREM_NB'].shape[0])

In [ ]:
# As noted above, cds_ds_db.ev_ami_final only contains data from 2022-04-01 to 2023-04-01!
# Therefore, if any search times outside of this, reset them (this should only be the case for EVs)
# I still want them to be set as close as possible to original values, so handle those before 2022-04-01
#   and those after 2023-04-01 separately (don't want all to be exactly the same, hence the +=2*pd.Timedelta(expand_time))
all_trff_dfs=all_trff_dfs.reset_index(drop=True)
#-----
t_search_min_col_idx = Utilities_df.find_idxs_in_highest_order_of_columns(all_trff_dfs, 't_search_min')
t_search_max_col_idx = Utilities_df.find_idxs_in_highest_order_of_columns(all_trff_dfs, 't_search_max')
assert(len(t_search_min_col_idx)==len(t_search_max_col_idx)==1)
t_search_min_col_idx = t_search_min_col_idx[0]
t_search_max_col_idx = t_search_max_col_idx[0]
#-----
for idx_i in range(all_trff_dfs.shape[0]):
    if all_trff_dfs.iloc[idx_i]['t_search_min']<pd.to_datetime('2022-04-01'):
        t_min_i, t_max_i = Utilities_dt.get_random_datetime_interval_between(
            EVS_t_min, 
            EVS_t_min+2*pd.Timedelta(expand_time), 
            pd.Timedelta(expand_time)
        ) 
        all_trff_dfs.iloc[idx_i, [t_search_min_col_idx, t_search_max_col_idx]] = t_min_i, t_max_i
    elif all_trff_dfs.iloc[idx_i]['t_search_max']>pd.to_datetime('2023-04-01'):
        t_min_i, t_max_i = Utilities_dt.get_random_datetime_interval_between(
            EVS_t_max-2*pd.Timedelta(expand_time), 
            EVS_t_max, 
            pd.Timedelta(expand_time)
        )
        all_trff_dfs.iloc[idx_i, [t_search_min_col_idx, t_search_max_col_idx]] = t_min_i, t_max_i
    else:
        continue
#-----
assert(
    all_trff_dfs[
        (all_trff_dfs['t_search_min']<pd.to_datetime('2022-04-01')) | 
        (all_trff_dfs['t_search_max']>pd.to_datetime('2023-04-01'))
    ].shape[0]==0
)

In [ ]:
print(f'#PNs:     {all_trff_dfs.shape[0]}')
print(f'#EVs:     {all_trff_dfs["EV"].sum()}')
print(f'#not EVs: {(all_trff_dfs["EV"]==0).sum()}')

In [ ]:
all_trff_dfs.to_pickle(r'C:\Users\s346557\Documents\LocalData\EVs\all_trff_dfs.pkl')

In [ ]:
start = time.time()
n_update=10
for i, (idx_i, row_i) in enumerate(all_trff_dfs.iterrows()):
    if i%n_update==0:
        print(f"{i}/{all_trff_dfs.shape[0]}")
    ami_i = AMINonVee(
        df_construct_type=DFConstructType.kRunSqlQuery, 
        build_sql_function_kwargs= dict(
            cols_of_interest = ['*'], 
            premise_nbs = row_i['PREM_NB'],  
            date_range = [
                row_i['t_search_min'], 
                row_i['t_search_max']
            ], 
            schema_name='cds_ds_db', 
            table_name='ev_ami_final'
        ), 
        init_df_in_constructor=True
    )
    df_i = ami_i.df.copy()
    df_i['opco'] = row_i['opco']
    save_name_i = f"{row_i['opco']}_{row_i['PREM_NB']}.csv"
    df_i.to_csv(os.path.join(save_dir_base, save_name_i), index=False)
    
print(time.time()-start)